# 🏷️ Shaket Protocol: Reverse Auction Demo

### Scenario
One buyer wants to purchase a power bank from 5 competing sellers using a reverse auction with Shaket.

### What is a Reverse Auction?

**Reverse Auction:**
- Multiple **sellers** compete to sell to one buyer
- Price goes **DOWN** ⬇️, service goes **UP** ⬆️
- Example: Government procurement, B2B purchasing

**Traditional Auction:**
- Multiple **buyers** compete to purchase from one seller
- Price goes **UP** ⬆️
- Example: Art auction, eBay bidding

# Part 1: Setting Up Agents

Now let's configure our agents. First, imports and setup:

In [1]:
import asyncio
import logging
import sys
from pathlib import Path
from threading import Thread
import uvicorn

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from src.core.types import Item, SessionType, AgentRole
from src.client import ShaketClient
from src.server import ShaketServer
from src.agents import SendOfferAction, SendDiscoveryAction

# Configure logging for clean output
logging.basicConfig(level=logging.WARNING, format="%(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Show only user-facing logs
logging.getLogger("src.coordinators.reverse_auction").setLevel(logging.INFO)
logging.getLogger("src.client").setLevel(logging.WARNING)
logging.getLogger("src.server").setLevel(logging.WARNING)
logging.getLogger("src.state.state_manager").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("a2a").setLevel(logging.WARNING)

print("✅ Imports complete")

✅ Imports complete


## 2.1 Define the Item

What are we buying? An **Anker PowerCore 20000mAh Power Bank**.

In [2]:
power_bank = Item(
    id="pb-anker-20k",
    name="Anker PowerCore 20000mAh Power Bank",
    description="High-capacity portable charger with dual USB ports",
    category="electronics",
    metadata={
        "brand": "Anker",
        "capacity": "20000mAh",
        "condition": "new",
    },
)

print("📦 Item Details:")
print(f"   Name: {power_bank.name}")
print(f"   Category: {power_bank.category}")
print(f"   Brand: {power_bank.metadata['brand']}")
print(f"   Condition: {power_bank.metadata['condition']}")

📦 Item Details:
   Name: Anker PowerCore 20000mAh Power Bank
   Category: electronics
   Brand: Anker
   Condition: new


## 2.2 Buyer Agent Setup

### What Does the Buyer Agent Do?

The buyer agent has a simple job:

1. **Collect offers** from all sellers each round
2. **Calculate and decide market info**
3. **Send feedback** to all sellers so they can adjust
4. **Present results** at the end for human decision

### Important!

**The buyer agent does NOT make decisions** - it just coordinates!

In [3]:
class SimpleBuyerAgent:
    """Buyer agent: collects offers from sellers and provides market feedback."""

    def __init__(self, target_price: float = 70):
        self.target_price = target_price

    async def decide_next_action(self, session_id: str, state):
        """
        Buyer sends market info to sellers after each round.
        """
        # Get offers from current round
        current_offers = state.offers_by_round.get(state.current_round, [])

        if current_offers:
            prices = [o.price for o in current_offers]
            min_price = min(prices)
            max_price = max(prices)
            avg_price = sum(prices) / len(prices)

            # Send market feedback to sellers
            return SendDiscoveryAction(
                message=f"Round {state.current_round} market info",
                discovery_data={
                    "round": state.current_round,
                    "min_offer": min_price,
                    "max_offer": max_price,
                    "avg_offer": avg_price,
                    "num_offers": len(current_offers),
                },
            )

        return SendDiscoveryAction(
            message="Waiting for offers", discovery_data={"round": state.current_round}
        )

print("✅ Buyer agent defined")
print("   Target price: $70")

✅ Buyer agent defined
   Target price: $70


## 2.3 Seller Agent Setup

### What Makes Seller Agents Interesting?

Each seller has a **strategy** for how they compete:

- **Aggressive**: Drop prices early and often to win quickly
- **Conservative**: Preserve profit margins, make small adjustments
- **Balanced**: Randomized competitive pricing
- **Last-minute**: Hold back early, then go all-in on the final round

In [4]:
class SimpleSellerAgent:
    """Seller agent: submits competitive offers based on market feedback."""

    def __init__(
        self,
        seller_id: str,
        initial_price: float,
        min_price: float,
        aggressiveness: float = 2.0,
        strategy: str = "balanced",
    ):
        self.seller_id = seller_id
        self.current_price = initial_price
        self.min_price = min_price
        self.aggressiveness = aggressiveness
        self.strategy = strategy

    def calculate_undercut_amount(self, current_round: int, total_rounds: int) -> float:
        """Calculate how much to undercut based on strategy and round."""
        import random

        is_final_round = current_round == total_rounds

        if self.strategy == "aggressive":
            return self.aggressiveness + random.uniform(0, 2)

        elif self.strategy == "conservative":
            return self.aggressiveness * 0.5 + random.uniform(0, 1)

        elif self.strategy == "last_minute":
            if is_final_round:
                return self.aggressiveness * 2.0 + random.uniform(0, 3)  # Very aggressive!
            else:
                return self.aggressiveness * 0.3 + random.uniform(0, 1)  # Conservative early

        else:  # "balanced"
            return self.aggressiveness + random.uniform(0, 3)

    async def decide_next_action(self, session_id: str, state):
        """Seller submits offers and adjusts based on market feedback."""
        current_round = state.current_round
        total_rounds = state.total_rounds

        if state.discovery_messages:
            last_discovery = state.discovery_messages[-1]
            market_data = last_discovery.get("data", {})
            round_num = market_data.get("round_number", current_round)
            current_round = round_num
            total_rounds = market_data.get("total_rounds", total_rounds)

            message_text = market_data.get("message", "")
            if "Lowest offer: $" in message_text:
                import re
                match = re.search(r"Lowest offer: \$(\d+\.?\d*)", message_text)
                if match:
                    min_offer = float(match.group(1))
                    undercut = self.calculate_undercut_amount(current_round, total_rounds)
                    new_price = max(min_offer - undercut, self.min_price)

                    if new_price < self.current_price:
                        old_price = self.current_price
                        self.current_price = new_price
                        reduction = old_price - self.current_price
                        logger.info(
                            f"   💚 {self.seller_id:9} [{self.strategy:12}] "
                            f"${old_price:6.2f} → ${self.current_price:6.2f} (-${reduction:.2f})"
                        )
                    else:
                        logger.info(
                            f"   💚 {self.seller_id:9} [{self.strategy:12}] "
                            f"${self.current_price:6.2f} (holding)"
                        )
                else:
                    logger.info(
                        f"   💚 {self.seller_id:9} [{self.strategy:12}] "
                        f"${self.current_price:6.2f} (initial)"
                    )
            else:
                logger.info(
                    f"   💚 {self.seller_id:9} [{self.strategy:12}] "
                    f"${self.current_price:6.2f} (initial)"
                )

        return SendOfferAction(
            price=self.current_price,
            message=f"{self.seller_id} offer for round {current_round}",
            metadata={"seller_id": self.seller_id},
        )

print("✅ Seller agent defined")

✅ Seller agent defined


## 2.4 Configure Our 5 Sellers

Let's create 5 sellers with different strategies and see how they compete!

In [5]:
sellers_config = [
    {
        "id": "Seller_A",
        "initial_price": 95.0,
        "min_price": 75.0,
        "aggressiveness": 2.5,
        "strategy": "aggressive",
    },
    {
        "id": "Seller_B",
        "initial_price": 90.0,
        "min_price": 70.0,
        "aggressiveness": 1.0,
        "strategy": "conservative",
    },
    {
        "id": "Seller_C",
        "initial_price": 88.0,
        "min_price": 72.0,
        "aggressiveness": 2.0,
        "strategy": "balanced",
    },
    {
        "id": "Seller_D",
        "initial_price": 92.0,
        "min_price": 78.0,
        "aggressiveness": 1.5,
        "strategy": "last_minute",
    },
    {
        "id": "Seller_E",
        "initial_price": 85.0,
        "min_price": 68.0,
        "aggressiveness": 3.0,
        "strategy": "aggressive",
    },
]

print("💚 Sellers Configuration:")
print("═" * 80)
print(f"{'Seller':<10} {'Initial':<10} {'Minimum':<10} {'Strategy':<15} {'Aggressiveness':<15}")
print("═" * 80)
for config in sellers_config:
    print(
        f"{config['id']:<10} ${config['initial_price']:<9.0f} ${config['min_price']:<9.0f} "
        f"{config['strategy']:<15} {config['aggressiveness']:<15}"
    )
print("═" * 80)
print("\n📊 Strategy Breakdown:")
print("   • Aggressive (A, E): Drop prices early to win quickly")
print("   • Conservative (B): Small adjustments to preserve profit")
print("   • Balanced (C): Randomized competitive pricing")
print("   • Last-minute (D): Hold back early, aggressive in final round")

💚 Sellers Configuration:
════════════════════════════════════════════════════════════════════════════════
Seller     Initial    Minimum    Strategy        Aggressiveness 
════════════════════════════════════════════════════════════════════════════════
Seller_A   $95        $75        aggressive      2.5            
Seller_B   $90        $70        conservative    1.0            
Seller_C   $88        $72        balanced        2.0            
Seller_D   $92        $78        last_minute     1.5            
Seller_E   $85        $68        aggressive      3.0            
════════════════════════════════════════════════════════════════════════════════

📊 Strategy Breakdown:
   • Aggressive (A, E): Drop prices early to win quickly
   • Conservative (B): Small adjustments to preserve profit
   • Balanced (C): Randomized competitive pricing
   • Last-minute (D): Hold back early, aggressive in final round


## 2.5 Start Seller Servers

Each seller runs as a separate **ShaketServer** instance.

These servers:
- Listen for auction invitations from buyers
- Process market feedback messages
- Submit competitive offers each round

In [6]:
def start_server_background(server):
    """Start a Shaket server in a background thread."""
    def run_server():
        uvicorn.run(
            server.app.build(), host=server.host, port=server.port, log_level="warning"
        )
    server_thread = Thread(target=run_server, daemon=True)
    server_thread.start()


# Create and start all seller servers
servers = []
for idx, config in enumerate(sellers_config):
    seller_agent = SimpleSellerAgent(
        seller_id=config["id"],
        initial_price=config["initial_price"],
        min_price=config["min_price"],
        aggressiveness=config["aggressiveness"],
        strategy=config["strategy"],
    )

    server = ShaketServer(
        name=config["id"],
        description=f"Power bank seller - {config['id']}",
        supported_session_types=[SessionType.REVERSE_AUCTION],
        supported_roles=[AgentRole.SELLER],
        reverse_auction_agent=seller_agent,
        host="localhost",
        port=8001 + idx,
    )

    start_server_background(server)
    servers.append(server)
    print(f"✅ Started {config['id']} on port {8001 + idx}")

await asyncio.sleep(2)
print("\n🎉 All 5 seller servers are running and ready!")

✅ Started Seller_A on port 8001
✅ Started Seller_B on port 8002
✅ Started Seller_C on port 8003
✅ Started Seller_D on port 8004
✅ Started Seller_E on port 8005

🎉 All 5 seller servers are running and ready!


## 2.6 Create Buyer Client

Now let's create the buyer client that will coordinate the auction:

In [7]:
# Create buyer agent
buyer_agent = SimpleBuyerAgent(target_price=70)

# Completion callback to display results
async def on_complete(result):
    print("\n" + "═" * 70)
    print("🏁 REVERSE AUCTION COMPLETE!")
    print("═" * 70)

    if result.data.get("success"):
        all_offers = result.data.get("all_offers", [])
        print(f"\n📊 Received {len(all_offers)} total offers across 3 rounds\n")

        # Show price range
        price_range = result.data.get("price_range", {})
        if price_range:
            print(
                f"💰 Price Range: ${price_range['min']:.2f} - ${price_range['max']:.2f}   "
                f"Average: ${price_range['avg']:.2f}\n"
            )

        # Find best (lowest) offer
        if all_offers:
            best_offer = min(all_offers, key=lambda o: o["price"])
            best_price = best_offer["price"]
            seller_id = best_offer.get("metadata", {}).get(
                "seller_id", "Unknown Seller"
            )
            print(f"🎯 BEST OFFER: ${best_price:.2f} from {seller_id}")
            print(f"\n💡 Note: Shaket presented all offers - YOU decide which to accept!")
    else:
        print("❌ No offers received")

    print("═" * 70 + "\n")


client = await ShaketClient.create(
    remote_agent_urls=[f"http://localhost:{8001+i}" for i in range(5)],
    reverse_auction_agent=buyer_agent,
    on_session_complete=on_complete,
)

print("✅ Buyer client created and ready to start auction")

✅ Buyer client created and ready to start auction


---

# Part 3: Running the Auction

## Let's Start the Auction!

The auction will run for **3 rounds** with **5 seconds per round**.

### What to Watch For:

- **Round 1**: All sellers submit initial prices
- **Round 2**: Sellers see market feedback and adjust
  - Aggressive sellers should drop significantly
  - Conservative sellers make small adjustments
  - Last-minute sellers hold back
- **Round 3**: Final competitive push
  - Last-minute sellers make their big move!
  - All sellers give their best final offer

Let's run it!

In [8]:
# Start the auction!
print("\n" + "═" * 70)
print("🚀 STARTING REVERSE AUCTION")
print("═" * 70)
print(f"\n📦 Item: {power_bank.name}")
print(f"💙 Buyer: Target price ${buyer_agent.target_price}\n")
print(f"💚 Sellers ({len(sellers_config)} competing):")
for config in sellers_config:
    print(
        f"   • {config['id']}: ${config['initial_price']:.0f} → ${config['min_price']:.0f} min "
        f"[{config['strategy']}]"
    )
print(f"\n🔄 Format: 3 rounds, 5 seconds each")
print(f"\n{'═'*70}\n")

result = await client.start_reverse_auction(
    counterparty_endpoints=[f"http://localhost:{8001+i}" for i in range(5)],
    item=power_bank,
    role="buyer",
    rounds=3,
    round_duration=5.0,  # 5 seconds per round
)

if not result["success"]:
    print(f"❌ Failed to start reverse auction: {result.get('error')}")

[ReverseAuctionCoordinator] Started session reverse-auction-1459f33bfbd7

🔄 Round 1/3
   💚 Seller_A  [aggressive  ] $ 95.00 (initial)
   💚 Seller_B  [conservative] $ 90.00 (initial)
   💚 Seller_C  [balanced    ] $ 88.00 (initial)
   💚 Seller_D  [last_minute ] $ 92.00 (initial)
   💚 Seller_E  [aggressive  ] $ 85.00 (initial)



══════════════════════════════════════════════════════════════════════
🚀 STARTING REVERSE AUCTION
══════════════════════════════════════════════════════════════════════

📦 Item: Anker PowerCore 20000mAh Power Bank
💙 Buyer: Target price $70

💚 Sellers (5 competing):
   • Seller_A: $95 → $75 min [aggressive]
   • Seller_B: $90 → $70 min [conservative]
   • Seller_C: $88 → $72 min [balanced]
   • Seller_D: $92 → $78 min [last_minute]
   • Seller_E: $85 → $68 min [aggressive]

🔄 Format: 3 rounds, 5 seconds each

══════════════════════════════════════════════════════════════════════




🔄 Round 2/3
   💚 Seller_B  [conservative] $ 90.00 → $ 83.58 (-$6.42)
   💚 Seller_C  [balanced    ] $ 88.00 → $ 82.46 (-$5.54)
   💚 Seller_D  [last_minute ] $ 92.00 → $ 84.06 (-$7.94)
   💚 Seller_A  [aggressive  ] $ 95.00 → $ 81.93 (-$13.07)
   💚 Seller_E  [aggressive  ] $ 85.00 → $ 80.53 (-$4.47)

🔄 Round 3/3
   💚 Seller_B  [conservative] $ 83.58 → $ 79.57 (-$4.02)
   💚 Seller_C  [balanced    ] $ 82.46 → $ 77.85 (-$4.61)
   💚 Seller_D  [last_minute ] $ 84.06 → $ 78.00 (-$6.06)
   💚 Seller_E  [aggressive  ] $ 80.53 → $ 77.10 (-$3.43)
   💚 Seller_A  [aggressive  ] $ 81.93 → $ 76.23 (-$5.71)



══════════════════════════════════════════════════════════════════════
🏁 REVERSE AUCTION COMPLETE!
══════════════════════════════════════════════════════════════════════

📊 Received 15 total offers across 3 rounds

💰 Price Range: $76.23 - $95.00   Average: $83.42

🎯 BEST OFFER: $76.23 from Seller_A

💡 Note: Shaket presented all offers - YOU decide which to accept!
══════════════════════════════════════════════════════════════════════



## 🎬 What Just Happened?

### Round-by-Round Analysis

**Round 1:**
- All sellers submitted their initial prices
- No market feedback yet - everyone showing their starting position

**Round 2:**
- Sellers received feedback: lowest offer, highest offer, average
- **Aggressive sellers** (A, E) dropped prices significantly
- **Conservative seller** (B) made small adjustments
- **Last-minute seller** (D) held back
- **Balanced seller** (C) adjusted competitively

**Round 3 - Final Round:**
- This is where the **last-minute strategy** shines!
- Seller D finally makes their aggressive move
- All sellers push to their limits

### Key Insights

✅ **Shaket coordinated** all communication between 1 buyer and 5 sellers  
✅ **State was managed** automatically with full audit trail  
✅ **Timing was handled** - each round ran for exactly 5 seconds  
✅ **No winner was picked** - all offers presented to buyer for decision  

---

# Bonus: Under the Hood

Want to see what Shaket tracked during the auction?

## Inspect Session State

In [ ]:
# Get the buyer's session state
client_sessions = client.state_manager.list_sessions()
if client_sessions:
    buyer_state = client_sessions[0]
    
    print("\n" + "─" * 70)
    print("📊 BUYER SESSION STATE")
    print("─" * 70)
    print(f"Session ID: {buyer_state.session_id}")
    print(f"Role: {buyer_state.role.value}")
    print(f"Status: {buyer_state.status}")
    print(f"Rounds: {buyer_state.current_round}/{buyer_state.total_rounds}")
    
    print(f"\n📈 Offers by Round:")
    for round_num in sorted(buyer_state.offers_by_round.keys()):
        offers = buyer_state.offers_by_round[round_num]
        print(f"\n  Round {round_num}: {len(offers)} offers")
        for offer in sorted(offers, key=lambda o: o.price):
            seller = offer.metadata.get('seller_id', 'Unknown')
            print(f"    ${offer.price:6.2f} - {seller}")
    
    print(f"\n✅ Total Offers Received: {len(buyer_state.all_offers)}")
    print("─" * 70)

## View Event History

Shaket tracks **every event** with timestamps for full auditability:

In [ ]:
if client_sessions:
    session_id = buyer_state.session_id
    events = client.state_manager.get_events(session_id)
    
    print("\n" + "─" * 70)
    print(f"📜 EVENT LOG ({len(events)} events)")
    print("─" * 70)
    
    for i, event in enumerate(events, 1):
        timestamp = event.timestamp.strftime("%H:%M:%S.%f")[:-3]
        event_info = f"{i:2}. [{timestamp}] {event.event_type.value}"
        
        if "offer" in event.data:
            offer_data = event.data["offer"]
            event_info += f" - ${offer_data.get('price', 'N/A')}"
        elif "round_number" in event.data:
            event_info += f" - round {event.data['round_number']}"
        
        print(event_info)
    
    print("─" * 70)
    print("\n💡 Every action is tracked - full transparency and auditability!")

---

# Key Takeaways

## What Shaket Does ✅

✅ **Coordinates** multi-round auctions with automatic timing and state management  
✅ **Routes messages** between buyer and multiple sellers using A2A protocol  
✅ **Tracks state** with full event history for auditability  
✅ **Provides structure** for agents to compete and negotiate systematically  

## What Shaket Does NOT Do ❌

❌ **Make decisions** - Your agent decides pricing strategy and tactics  
❌ **Pick winners** - The buyer/user makes the final choice  
❌ **Handle payments** - Shaket is coordination, not transactions  
❌ **Enforce rules** - Your agents control their own behavior  

## The Value Proposition 💡

> ### **"You bring the intelligence, Shaket handles coordination"**

- 🔌 **Plug in any agent framework** - LangChain, CrewAI, custom agents, LLMs
- 🏗️ **Shaket provides the market infrastructure** - messaging, timing, state
- 🧠 **Your agents make strategic decisions** - pricing, negotiation tactics
- 🌐 **Works with A2A protocol** - interoperability across frameworks

---

## Next Steps

### Try Modifying:

- 🎯 **Seller strategies**: Create new bidding algorithms
- 🔄 **Number of rounds**: More rounds = more price discovery
- 💰 **Pricing parameters**: Adjust initial prices and minimums
- 📦 **Item metadata**: Add quality scores, shipping costs, warranties
- ⏱️ **Round duration**: Speed up or slow down the auction

### Explore Other Examples:

- 🤝 **`powerbank_negotiation.py`** - 1-on-1 negotiation between buyer and seller
- 🤖 **`powerbank_negotiation_llm.py`** - LLM-powered intelligent negotiation

---

## Learn More

- 📖 **GitHub**: [github.com/shaketlabs/shaket](https://github.com/shaketlabs/shaket)
- 🌐 **A2A Protocol**: [a2a-protocol.org](https://a2a-protocol.org)
- 📚 **Documentation**: Check the `/docs` folder in the repository

---

**Thank you for trying Shaket!** 🎉